<a href="https://colab.research.google.com/github/yesstarr/2024k-hackerton_COSMICA/blob/main/ai_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import requests
import pandas as pd
import folium
import json

In [ ]:
class MF():
    def __init__(self, ratings, hyper_params):
        self.R = np.array(ratings)
        # 출발지 수(num_starts)와 목적지 수(num_destinations)를 받아온다.
        self.num_starts, self.num_destinations = np.shape(self.R)

        # 하이퍼파라미터 설정
        self.K = hyper_params['K']
        self.alpha = hyper_params['alpha']
        self.beta = hyper_params['beta']
        self.iterations = hyper_params['iterations']
        self.verbose = hyper_params['verbose']

        # 출발지와 목적지 인덱스 매핑 작업
        start_id_index = []
        index_start_id = []
        for i, one_id in enumerate(ratings['start_location'].unique()):
            start_id_index.append([one_id, i])
            index_start_id.append([i, one_id])
        self.start_id_index = dict(start_id_index)
        self.index_start_id = dict(index_start_id)

        destination_id_index = []
        index_destination_id = []
        for i, one_id in enumerate(ratings['destination_location'].unique()):
            destination_id_index.append([one_id, i])
            index_destination_id.append([i, one_id])
        self.destination_id_index = dict(destination_id_index)
        self.index_destination_id = dict(index_destination_id)

    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []

        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)

        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors ** 2))

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            # 출발지와 목적지에 대한 경향성 업데이트
            self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
            self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

            # P 행렬과 Q 행렬 업데이트
            self.P[i, :] += self.alpha * ((e * self.Q[j, :]) - (self.beta * self.P[i, :]))
            self.Q[j, :] += self.alpha * ((e * self.P[i, :]) - (self.beta * self.Q[j, :]))

    def get_prediction(self, i, j):  # 경로 추천 점수 예측 함수
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # 테스트 세트 설정
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            x = self.start_id_index[ratings_test.iloc[i, 0]]  # 출발지 ID
            y = self.destination_id_index[ratings_test.iloc[i, 1]]  # 목적지 ID
            z = ratings_test.iloc[i, 2]  # 실제 추천 점수
            test_set.append([x, y, z])
            self.R[x, y] = 0  # 트레이닝에서 제외
        self.test_set = test_set
        return test_set

    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error / len(self.test_set))

    def test(self):
        self.P = np.random.normal(scale=1. / self.K, size=(self.num_starts, self.K))
        self.Q = np.random.normal(scale=1. / self.K, size=(self.num_destinations, self.K))
        self.b_u = np.zeros(self.num_starts)
        self.b_d = np.zeros(self.num_destinations)
        self.b = np.mean(self.R[self.R.nonzero()])

        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i + 1, rmse1, rmse2))
            if self.verbose:
                if (i + 1) % 10 == 0:
                    print('Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f' % (i + 1, rmse1, rmse2))
        return training_process

    def get_one_prediction(self, start_id, destination_id):
        return self.get_prediction(self.start_id_index[start_id], self.destination_id_index[destination_id])

    def full_prediction(self):
        return self.b + self.b_u[:, np.newaxis] + self.b_d[np.newaxis, :] + self.P.dot(self.Q.T)


R_temp = ratings.pivot(index='start_location', columns='destination_location', values='rating').fillna(0)
hyper_params = {
     'K': 30,
     'alpha': 0.001,
     'beta': 0.02,
     'iterations': 100,
     'verbose': True
}
mf = MF(R_temp, hyper_params)
test_set = mf.set_test(ratings_test)
result = mf.test()


In [ ]:
cbnu_lat = 36.632473380701
cbnu_long = 127.45314301376
library_lat = 36.62844389404879
library_long = 127.45745658874512

In [ ]:
cbnu_location = folium.Map(location = [cbnu_lat,cbnu_long], zoom_start =16)
library_location = folium.Map(location = [library_lat,library_long],zoom_start = 16)

In [ ]:
import numpy as np
import json
from math import radians, cos, sin, sqrt, atan2
from decimal import Decimal
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# JSON 데이터를 행렬로 변환하는 함수
def json_to_matrix(json_data):
    data = json.loads(json_data)
    points = [(Decimal(point["lat"]), Decimal(point["long"])) for point in data["data"]]

    num_points = len(points)
    distance_matrix = np.zeros((num_points, num_points))

    # 각 위치 사이의 거리를 계산하여 행렬에 저장
    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                distance_matrix[i][j] = haversine(points[i][0], points[i][1], points[j][0], points[j][1])

    return distance_matrix
#출발지 첫번째 lat,long 이고 도착지 마지막 lat,long이라 가정
json_data_list=[
    '''
    {
      "data" : [
         {"lat" :"36.632473380701","long" : "127.45314301376"},
         {"lat" :"36.632473380705","long" : "127.45314301377"},
         {"lat" :"36.632473380706","long" : "127.45314301378"},
         {"lat" :"36.632473380707","long" : "127.45314301379"},
         {"lat" :"36.632473380708","long" : "127.45314301380"}
      ]
    }
    ''',
    '''
    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380705","long" : "127.45314301377"},
          {"lat" :"36.632473380706","long" : "127.45314301378"},
          {"lat" :"36.632473380707","long" : "127.45314301379"},
          {"lat" :"36.632473380708","long" : "127.45314301380"}
      ]
    }
    ''',
    '''
    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380712","long" : "127.45314301379"},
          {"lat" :"36.632473380713","long" : "127.45314301380"},
          {"lat" :"36.632473380714","long" : "127.45314301381"},
          {"lat" :"36.632473380708","long" : "127.45314301380"}
      ]
    }
    '''
]
distance_matrices = []
for json_data in json_data_list:
    distance_matrix = json_to_matrix(json_data)
    distance_matrices.append(distance_matrix)

# 각 행렬 확인
for i, matrix in enumerate(distance_matrices):
    print(f"Distance matrix {i+1}:")
    print(matrix)

Distance matrix 1:
[[0.00000000e+00 9.97024353e-10 1.86923035e-09 2.75883533e-09
  3.65315048e-09]
 [9.97024353e-10 0.00000000e+00 8.99217929e-10 1.79843586e-09
  2.69765379e-09]
 [1.86923035e-09 8.99217929e-10 0.00000000e+00 8.99217929e-10
  1.79843586e-09]
 [2.75883533e-09 1.79843586e-09 8.99217929e-10 0.00000000e+00
  8.99217929e-10]
 [3.65315048e-09 2.69765379e-09 1.79843586e-09 8.99217929e-10
  0.00000000e+00]]
Distance matrix 2:
[[0.00000000e+00 9.97024353e-10 1.86923035e-09 2.75883533e-09
  3.65315048e-09]
 [9.97024353e-10 0.00000000e+00 8.99217929e-10 1.79843586e-09
  2.69765379e-09]
 [1.86923035e-09 8.99217929e-10 0.00000000e+00 8.99217929e-10
  1.79843586e-09]
 [2.75883533e-09 1.79843586e-09 8.99217929e-10 0.00000000e+00
  8.99217929e-10]
 [3.65315048e-09 2.69765379e-09 1.79843586e-09 8.99217929e-10
  0.00000000e+00]]
Distance matrix 3:
[[0.00000000e+00 2.94315118e-09 3.81052727e-09 4.68991290e-09
  3.65315048e-09]
 [2.94315118e-09 0.00000000e+00 8.99217929e-10 1.79843586e-09

In [ ]:
class MF():
    def __init__(self, distance_matrix, hyper_params):
        self.R = np.array(distance_matrix)
        self.num_starts, self.num_destinations = np.shape(self.R)

        # 하이퍼파라미터 설정
        self.K = hyper_params['K']
        self.alpha = hyper_params['alpha']
        self.beta = hyper_params['beta']
        self.iterations = hyper_params['iterations']
        self.verbose = hyper_params['verbose']

        # 거리 기반 최적 경로 예측을 위한 행렬 초기화
        self.P = np.random.normal(scale=1. / self.K, size=(self.num_starts, self.K))
        self.Q = np.random.normal(scale=1. / self.K, size=(self.num_destinations, self.K))
        self.b_u = np.zeros(self.num_starts)
        self.b_d = np.zeros(self.num_destinations)
        self.b = np.mean(self.R[self.R.nonzero()])

        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []

        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)

        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors ** 2))

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
            self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

            self.P[i, :] += self.alpha * ((e * self.Q[j, :]) - (self.beta * self.P[i, :]))
            self.Q[j, :] += self.alpha * ((e * self.P[i, :]) - (self.beta * self.Q[j, :]))

    def test(self):
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            training_process.append((i + 1, rmse1))
            if self.verbose and (i + 1) % 10 == 0:
                print(f'Iteration: {i + 1}; Train RMSE = {rmse1:.4f}')
        return training_process
# 하이퍼파라미터 설정
hyper_params = {
    'K': 30,
    'alpha': 0.001,
    'beta': 0.02,
    'iterations': 100,
    'verbose': True
}

# MF 모델 생성 및 학습
mf = MF(distance_matrix, hyper_params)
result = mf.test()


Iteration: 10; Train RMSE = 0.0070
Iteration: 20; Train RMSE = 0.0070
Iteration: 30; Train RMSE = 0.0069
Iteration: 40; Train RMSE = 0.0068
Iteration: 50; Train RMSE = 0.0067
Iteration: 60; Train RMSE = 0.0067
Iteration: 70; Train RMSE = 0.0066
Iteration: 80; Train RMSE = 0.0065
Iteration: 90; Train RMSE = 0.0065
Iteration: 100; Train RMSE = 0.0064


In [ ]:
import numpy as np
import json
from math import radians, cos, sin, sqrt, atan2

# Haversine 함수로 두 좌표 사이의 거리 계산
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# JSON 데이터를 행렬로 변환하는 함수
def json_to_matrix(json_data):
    data = json.loads(json_data)
    points = [(float(point["lat"]), float(point["long"])) for point in data["data"]]

    num_points = len(points)
    distance_matrix = np.zeros((num_points, num_points))

    # 각 위치 사이의 거리를 계산하여 행렬에 저장
    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                distance_matrix[i][j] = haversine(points[i][0], points[i][1], points[j][0], points[j][1])

    return distance_matrix

# Matrix Factorization 클래스
class MF():
    def __init__(self, distance_matrix, hyper_params):
        self.R = np.array(distance_matrix)
        self.num_starts, self.num_destinations = np.shape(self.R)

        # 하이퍼파라미터 설정
        self.K = hyper_params['K']
        self.alpha = hyper_params['alpha']
        self.beta = hyper_params['beta']
        self.iterations = hyper_params['iterations']
        self.verbose = hyper_params['verbose']

        # 거리 기반 최적 경로 예측을 위한 행렬 초기화
        self.P = np.random.normal(scale=1. / self.K, size=(self.num_starts, self.K))
        self.Q = np.random.normal(scale=1. / self.K, size=(self.num_destinations, self.K))
        self.b_u = np.zeros(self.num_starts)
        self.b_d = np.zeros(self.num_destinations)
        self.b = np.mean(self.R[self.R.nonzero()])

        rows, columns = self.R.nonzero()
        self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, columns)]

    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []

        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)

        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors ** 2))

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - (self.beta * self.b_u[i]))
            self.b_d[j] += self.alpha * (e - (self.beta * self.b_d[j]))

            self.P[i, :] += self.alpha * ((e * self.Q[j, :]) - (self.beta * self.P[i, :]))
            self.Q[j, :] += self.alpha * ((e * self.P[i, :]) - (self.beta * self.Q[j, :]))

    def test(self):
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            training_process.append((i + 1, rmse1))
            if self.verbose and (i + 1) % 10 == 0:
                print(f'Iteration: {i + 1}; Train RMSE = {rmse1:.4f}')
        return training_process

# 하이퍼파라미터 설정
hyper_params = {
    'K': 30,
    'alpha': 0.001,
    'beta': 0.02,
    'iterations': 700,
    'verbose': False  # 중간 출력 생략
}

# JSON 데이터 리스트
json_data_list=[
    '''
    {
      "data" : [
         {"lat" :"36.632473380701","long" : "127.45314301376"},
         {"lat" :"36.632473380705","long" : "127.45314301377"},
         {"lat" :"36.632473380706","long" : "127.45314301378"},
         {"lat" :"36.632473380707","long" : "127.45314301379"},
         {"lat" :"36.632473380708","long" : "127.45314301380"}
      ]
    }
    ''',
    '''
    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380705","long" : "127.45314301377"},
          {"lat" :"36.632473380706","long" : "127.45314301378"},
          {"lat" :"36.632473380707","long" : "127.45314301379"},
          {"lat" :"36.632473380708","long" : "127.45314301380"}
      ]
    }
    ''',
    '''
    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380712","long" : "127.45314301379"},
          {"lat" :"36.632473380713","long" : "127.45314301380"},
          {"lat" :"36.632473380714","long" : "127.45314301381"},
          {"lat" :"36.632473380708","long" : "127.45314301380"}
      ]
    }
    '''
]

# 각 데이터에 대해 MF 학습 및 RMSE 계산
best_rmse = float('inf')
best_json_data = None

for i, json_data in enumerate(json_data_list):
    # 거리 행렬 변환
    distance_matrix = json_to_matrix(json_data)

    # MF 모델 생성 및 학습
    mf = MF(distance_matrix, hyper_params)
    result = mf.test()

    # 최종 RMSE 추출
    final_rmse = result[-1][1]
    print(f"JSON Data {i+1}: RMSE = {final_rmse:.4f}")

    # 최저 RMSE 찾기
    if final_rmse < best_rmse:
        best_rmse = final_rmse
        best_json_data = json_data

# 가장 낮은 RMSE를 가진 데이터 출력
print("\nBest JSON Data with lowest RMSE:")
print(best_json_data)


JSON Data 1: RMSE = 0.0030
JSON Data 2: RMSE = 0.0033
JSON Data 3: RMSE = 0.0023

Best JSON Data with lowest RMSE:

    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380712","long" : "127.45314301379"},
          {"lat" :"36.632473380713","long" : "127.45314301380"},
          {"lat" :"36.632473380714","long" : "127.45314301381"},
          {"lat" :"36.632473380708","long" : "127.45314301380"}
      ]
    }
    


In [ ]:
import numpy as np
import json
from math import radians, cos, sin, sqrt, atan2
from itertools import permutations

# Haversine 함수를 사용해 두 좌표 사이의 거리 계산
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (km)
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance



# Brute-force TSP 경로 최적화
def tsp_bruteforce(distance_matrix):
    num_points = len(distance_matrix)
    all_routes = permutations(range(num_points))
    min_distance = float('inf')
    best_route = None

    for route in all_routes:
        current_distance = 0
        for i in range(len(route) - 1):
            current_distance += distance_matrix[route[i]][route[i + 1]]
        current_distance += distance_matrix[route[-1]][route[0]]  # 시작점으로 돌아가기

        if current_distance < min_distance:
            min_distance = current_distance
            best_route = route

    return min_distance, best_route

# Matrix Factorization
class MF():
    def __init__(self, R, hyper_params):
        self.R = np.array(R)
        self.num_users, self.num_items = np.shape(self.R)

        self.K = hyper_params['K']
        self.alpha = hyper_params['alpha']
        self.beta = hyper_params['beta']
        self.iterations = hyper_params['iterations']
        self.verbose = hyper_params['verbose']

        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        rows, cols = self.R.nonzero()
        self.samples = [(i, j, self.R[i, j]) for i, j in zip(rows, cols)]

    # RMSE 계산
    def rmse(self):
        xs, ys = self.R.nonzero()
        error = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            error.append(self.R[x, y] - prediction)
        return np.sqrt(np.mean(np.array(error)**2))

    # 예측 계산
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # 학습 과정
    def train(self):
        for it in range(self.iterations):
            np.random.shuffle(self.samples)
            for i, j, r in self.samples:
                prediction = self.get_prediction(i, j)
                e = r - prediction

                self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
                self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

                self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i, :])
                self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j, :])

            rmse = self.rmse()

    # 모든 경로에 대해 예측 점수를 반환하는 함수
    def recommend(self):
        predictions = np.zeros((self.num_users, self.num_items))
        for i in range(self.num_users):
            for j in range(self.num_items):
                predictions[i, j] = self.get_prediction(i, j)
        return predictions

# 경로 데이터를 평가 매트릭스로 변환하고 MF 적용
json_data_list = [
     '''
    {
      "data" : [
         {"lat" :"36.632473380701","long" : "127.45314301376"},
         {"lat" :"36.632473380705","long" : "127.45314301387"},
         {"lat" :"36.632473380706","long" : "127.45314301398"},
         {"lat" :"36.632473380707","long" : "127.45314301399"},
         {"lat" :"36.632473380708","long" : "127.45314301400"}
      ]
    }
    ''',
    '''
    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380705","long" : "127.45314301377"},
          {"lat" :"36.632473380706","long" : "127.45314301378"},
          {"lat" :"36.632473380707","long" : "127.45314301379"},
          {"lat" :"36.632473380708","long" : "127.45314301400"}
      ]
    }
    ''',
    '''
    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380712","long" : "127.45314301379"},
          {"lat" :"36.632473380713","long" : "127.45314301380"},
          {"lat" :"36.632473380714","long" : "127.45314301381"},
          {"lat" :"36.632473380708","long" : "127.45314301400"}
      ]
    }
    '''
]

# 평가 매트릭스
R = [
    [3, 3, 3],
    [3, 3, 3],
    [3, 3, 3]
]

# 하이퍼파라미터 설정 및 MF 모델 생성
hyper_params = {'K': 10, 'alpha': 0.01, 'beta': 0.01, 'iterations': 600, 'verbose': True}
mf = MF(R, hyper_params)
mf.train()

# 경로 추천
predicted_scores = mf.recommend()

# 가장 높은 점수를 가진 경로 추천
best_route_index = np.argmax(np.mean(predicted_scores, axis=0))
print(f"추천경로: {best_route_index + 1}번째 경로")
recommended_json_data = json_data_list[best_route_index]
print(recommended_json_data)

추천경로: 3번째 경로

    {
      "data" : [
          {"lat" :"36.632473380701","long" : "127.45314301376"},
          {"lat" :"36.632473380712","long" : "127.45314301379"},
          {"lat" :"36.632473380713","long" : "127.45314301380"},
          {"lat" :"36.632473380714","long" : "127.45314301381"},
          {"lat" :"36.632473380708","long" : "127.45314301400"}
      ]
    }
    


False